# **11 - Advanced Patterns: Expert-Level LangChain Techniques**

## **Overview**
In this notebook, we'll explore advanced patterns and techniques for building sophisticated AI applications. You'll learn about routing, fallbacks, retries, conditional logic, and complex orchestration patterns.

## **Learning Objectives**
By the end of this notebook, you will be able to:
- Implement dynamic routing based on input
- Build fallback chains for reliability
- Create retry mechanisms with exponential backoff
- Design conditional workflows
- Implement map-reduce patterns
- Build self-correcting systems

## **Prerequisites**
- Completion of notebooks 01-10
- Strong understanding of chains and agents
- Familiarity with async programming

## **Back-and-Forth Teaching Pattern**
This notebook follows our pattern:
1. **Instructor Activity**: Demonstrates a concept with complete examples
2. **Learner Activity**: You apply the concept with guidance and hidden solutions

## **Setup**

Let's install and import the necessary libraries:

In [ ]:
# Install required packages
!pip install langchain langchain-openai tenacity pydantic

In [ ]:
import os
from typing import Any, Dict, List, Optional, Union, Callable
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser
from langchain.schema.runnable import (
    Runnable, RunnablePassthrough, RunnableLambda, 
    RunnableBranch, RunnableParallel
)
from langchain.callbacks import get_openai_callback
from pydantic import BaseModel, Field, validator
from tenacity import retry, stop_after_attempt, wait_exponential
import time
import random
from enum import Enum
import json
import warnings
warnings.filterwarnings('ignore')

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

---

## **Instructor Activity 1: Routing and Conditional Logic**

Let's explore advanced routing patterns:

In [ ]:
# Dynamic routing based on input classification

class QueryType(str, Enum):
    TECHNICAL = "technical"
    CREATIVE = "creative"
    ANALYTICAL = "analytical"
    GENERAL = "general"

def classify_query(query: str) -> QueryType:
    """Classify the query type."""
    query_lower = query.lower()
    
    # Simple keyword-based classification
    technical_keywords = ["code", "debug", "api", "function", "algorithm", "error"]
    creative_keywords = ["story", "poem", "creative", "imagine", "design", "art"]
    analytical_keywords = ["analyze", "compare", "evaluate", "assess", "data", "statistics"]
    
    if any(keyword in query_lower for keyword in technical_keywords):
        return QueryType.TECHNICAL
    elif any(keyword in query_lower for keyword in creative_keywords):
        return QueryType.CREATIVE
    elif any(keyword in query_lower for keyword in analytical_keywords):
        return QueryType.ANALYTICAL
    else:
        return QueryType.GENERAL

# Create specialized chains for each type
technical_prompt = ChatPromptTemplate.from_template(
    """You are a technical expert. Provide detailed, accurate technical information.
    Use code examples when appropriate.
    
    Question: {query}
    Technical Answer:"""
)

creative_prompt = ChatPromptTemplate.from_template(
    """You are a creative writer. Be imaginative and engaging.
    Use vivid descriptions and creative language.
    
    Request: {query}
    Creative Response:"""
)

analytical_prompt = ChatPromptTemplate.from_template(
    """You are a data analyst. Provide structured, logical analysis.
    Break down complex topics systematically.
    
    Topic: {query}
    Analysis:"""
)

general_prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant. Provide a clear, concise answer.
    
    Question: {query}
    Answer:"""
)

# Create router using RunnableBranch
def create_router():
    llm = ChatOpenAI(temperature=0.7)
    
    # Define routing logic
    def route(inputs: Dict) -> Runnable:
        query_type = classify_query(inputs["query"])
        
        if query_type == QueryType.TECHNICAL:
            return technical_prompt | llm
        elif query_type == QueryType.CREATIVE:
            return creative_prompt | llm
        elif query_type == QueryType.ANALYTICAL:
            return analytical_prompt | llm
        else:
            return general_prompt | llm
    
    # Create router with RunnableLambda
    router = RunnableLambda(route)
    
    return router

# Test the router
router = create_router()

test_queries = [
    "Write a function to sort a list in Python",
    "Create a poem about the ocean",
    "Analyze the pros and cons of remote work",
    "What's the capital of France?"
]

for query in test_queries:
    query_type = classify_query(query)
    print(f"\nQuery: {query}")
    print(f"Type: {query_type}")
    
    # Route and get response
    response = router.invoke({"query": query})
    print(f"Response: {response.content[:200]}...\n")
    print("-" * 50)

In [ ]:
# Conditional chains with complex branching

class ConditionalChain:
    """Chain with conditional execution based on intermediate results."""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.7)
        self.setup_chains()
    
    def setup_chains(self):
        # Initial analysis chain
        self.analysis_chain = (
            ChatPromptTemplate.from_template(
                """Analyze this request and determine its complexity.
                Rate from 1-10 and explain why.
                
                Request: {input}
                
                Return JSON: {{"complexity": <number>, "reason": "<explanation>"}}"""
            )
            | self.llm
        )
        
        # Simple response chain
        self.simple_chain = (
            ChatPromptTemplate.from_template(
                """Provide a simple, straightforward answer.
                Request: {input}
                Simple Answer:"""
            )
            | self.llm
        )
        
        # Detailed response chain
        self.detailed_chain = (
            ChatPromptTemplate.from_template(
                """Provide a comprehensive, detailed response with examples.
                Request: {input}
                Detailed Answer:"""
            )
            | self.llm
        )
        
        # Expert response chain
        self.expert_chain = (
            ChatPromptTemplate.from_template(
                """Provide an expert-level analysis with advanced concepts,
                edge cases, and best practices.
                Request: {input}
                Expert Analysis:"""
            )
            | self.llm
        )
    
    def process(self, user_input: str) -> Dict:
        """Process input with conditional logic."""
        # Step 1: Analyze complexity
        analysis = self.analysis_chain.invoke({"input": user_input})
        
        # Parse complexity
        try:
            import json
            analysis_data = json.loads(analysis.content)
            complexity = analysis_data.get("complexity", 5)
        except:
            complexity = 5  # Default to medium
        
        # Step 2: Route based on complexity
        if complexity <= 3:
            response_type = "simple"
            response = self.simple_chain.invoke({"input": user_input})
        elif complexity <= 7:
            response_type = "detailed"
            response = self.detailed_chain.invoke({"input": user_input})
        else:
            response_type = "expert"
            response = self.expert_chain.invoke({"input": user_input})
        
        return {
            "input": user_input,
            "complexity": complexity,
            "response_type": response_type,
            "response": response.content
        }

# Test conditional chain
conditional = ConditionalChain()

test_inputs = [
    "What is 2+2?",
    "Explain how neural networks work",
    "Design a distributed system for real-time data processing with fault tolerance"
]

for input_text in test_inputs:
    result = conditional.process(input_text)
    print(f"\nInput: {result['input']}")
    print(f"Complexity: {result['complexity']}/10")
    print(f"Response Type: {result['response_type']}")
    print(f"Response: {result['response'][:200]}...\n")
    print("=" * 50)

---

## **Learner Activity 1: Build an Intelligent Router**

Create an advanced routing system that handles multiple dimensions.

**Task**: Build a router that considers:
1. Query type (technical, business, creative)
2. User expertise level (beginner, intermediate, expert)
3. Response format preference (brief, detailed, structured)
4. Language complexity adjustment
5. Fallback routing for ambiguous queries

Requirements:
- Multi-dimensional routing logic
- Adaptive response generation
- Graceful handling of edge cases
- Performance optimization

In [ ]:
# Build your intelligent router

class UserContext(BaseModel):
    expertise_level: str = Field(default="intermediate")
    format_preference: str = Field(default="detailed")
    # TODO: Add more context fields

class IntelligentRouter:
    def __init__(self):
        # TODO: Initialize routing components
        pass
    
    def analyze_query(self, query: str) -> Dict:
        """Analyze query across multiple dimensions."""
        # TODO: Implement multi-dimensional analysis
        pass
    
    def select_chain(self, analysis: Dict, context: UserContext) -> Runnable:
        """Select appropriate chain based on analysis and context."""
        # TODO: Implement selection logic
        pass
    
    def route(self, query: str, context: UserContext) -> str:
        """Route query to appropriate handler."""
        # TODO: Implement complete routing flow
        pass

# TODO: Test your router
# Test with various query types and user contexts
# Verify routing decisions

# Your test code here

In [ ]:
# Solution (hidden by default)

"""
from dataclasses import dataclass
from enum import Enum

class ExpertiseLevel(str, Enum):
    BEGINNER = "beginner"
    INTERMEDIATE = "intermediate"
    EXPERT = "expert"

class FormatPreference(str, Enum):
    BRIEF = "brief"
    DETAILED = "detailed"
    STRUCTURED = "structured"

@dataclass
class UserContext:
    expertise_level: ExpertiseLevel = ExpertiseLevel.INTERMEDIATE
    format_preference: FormatPreference = FormatPreference.DETAILED
    language: str = "en"
    max_tokens: int = 500

class IntelligentRouter:
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.7)
        self.analysis_cache = {}  # Cache analysis results
        self._setup_chains()
    
    def _setup_chains(self):
        '''Setup all chain variants.'''
        # Create chain templates for different combinations
        self.chain_templates = {}
        
        # Technical chains
        for level in ExpertiseLevel:
            for format_pref in FormatPreference:
                key = f"technical_{level.value}_{format_pref.value}"
                
                if level == ExpertiseLevel.BEGINNER:
                    complexity = "Use simple terms, avoid jargon, provide analogies."
                elif level == ExpertiseLevel.INTERMEDIATE:
                    complexity = "Use standard technical terms with brief explanations."
                else:
                    complexity = "Use advanced concepts, assume technical knowledge."
                
                if format_pref == FormatPreference.BRIEF:
                    format_inst = "Provide a concise answer in 2-3 sentences."
                elif format_pref == FormatPreference.DETAILED:
                    format_inst = "Provide a comprehensive answer with examples."
                else:
                    format_inst = "Structure your answer with clear sections and bullet points."
                
                template = f'''You are a technical assistant.
                {complexity}
                {format_inst}
                
                Question: {{query}}
                
                Answer:'''
                
                self.chain_templates[key] = ChatPromptTemplate.from_template(template) | self.llm
        
        # Business chains (similar structure)
        for level in ExpertiseLevel:
            key = f"business_{level.value}_detailed"
            template = f'''You are a business consultant.
            Expertise level: {level.value}
            Provide practical business insights.
            
            Question: {{query}}
            
            Business Perspective:'''
            
            self.chain_templates[key] = ChatPromptTemplate.from_template(template) | self.llm
        
        # Creative chains
        self.chain_templates["creative_any_any"] = (
            ChatPromptTemplate.from_template(
                '''You are a creative assistant.
                Be imaginative and engaging.
                
                Request: {query}
                
                Creative Response:'''
            ) | self.llm
        )
        
        # Fallback chain
        self.fallback_chain = (
            ChatPromptTemplate.from_template(
                '''Provide a helpful response to this query.
                Query: {query}
                Response:'''
            ) | self.llm
        )
    
    def analyze_query(self, query: str) -> Dict:
        '''Analyze query across multiple dimensions.'''
        # Check cache
        cache_key = hash(query)
        if cache_key in self.analysis_cache:
            return self.analysis_cache[cache_key]
        
        analysis_prompt = ChatPromptTemplate.from_template(
            '''Analyze this query and return JSON:
            {{
                "domain": "technical/business/creative/general",
                "complexity": 1-10,
                "requires_code": true/false,
                "requires_examples": true/false,
                "estimated_response_length": "short/medium/long"
            }}
            
            Query: {query}
            
            JSON:'''
        )
        
        try:
            response = (analysis_prompt | self.llm).invoke({"query": query})
            analysis = json.loads(response.content)
        except:
            # Fallback analysis
            analysis = self._simple_analysis(query)
        
        # Cache result
        self.analysis_cache[cache_key] = analysis
        return analysis
    
    def _simple_analysis(self, query: str) -> Dict:
        '''Simple keyword-based analysis as fallback.'''
        query_lower = query.lower()
        
        domain = "general"
        if any(word in query_lower for word in ["code", "function", "api", "debug"]):
            domain = "technical"
        elif any(word in query_lower for word in ["business", "market", "strategy", "roi"]):
            domain = "business"
        elif any(word in query_lower for word in ["create", "imagine", "story", "design"]):
            domain = "creative"
        
        return {
            "domain": domain,
            "complexity": len(query.split()) // 5 + 3,  # Rough estimate
            "requires_code": "code" in query_lower,
            "requires_examples": "example" in query_lower or "how" in query_lower,
            "estimated_response_length": "medium"
        }
    
    def select_chain(self, analysis: Dict, context: UserContext) -> Runnable:
        '''Select appropriate chain based on analysis and context.'''
        domain = analysis.get("domain", "general")
        complexity = analysis.get("complexity", 5)
        
        # Adjust expertise level based on complexity
        if complexity > 7 and context.expertise_level == ExpertiseLevel.BEGINNER:
            # Upgrade to intermediate for complex queries
            effective_level = ExpertiseLevel.INTERMEDIATE
        else:
            effective_level = context.expertise_level
        
        # Build chain key
        chain_key = f"{domain}_{effective_level.value}_{context.format_preference.value}"
        
        # Get chain or fallback
        if chain_key in self.chain_templates:
            return self.chain_templates[chain_key]
        
        # Try without format preference
        chain_key_simple = f"{domain}_{effective_level.value}_detailed"
        if chain_key_simple in self.chain_templates:
            return self.chain_templates[chain_key_simple]
        
        # Ultimate fallback
        return self.fallback_chain
    
    def route(self, query: str, context: UserContext = None) -> Dict:
        '''Route query to appropriate handler.'''
        if context is None:
            context = UserContext()
        
        start_time = time.time()
        
        # Analyze query
        analysis = self.analyze_query(query)
        
        # Select and execute chain
        chain = self.select_chain(analysis, context)
        
        try:
            response = chain.invoke({"query": query})
            response_text = response.content
        except Exception as e:
            # Fallback on error
            response_text = self.fallback_chain.invoke({"query": query}).content
            analysis["fallback_used"] = True
        
        # Post-process based on context
        if context.max_tokens and len(response_text) > context.max_tokens * 4:  # Rough char estimate
            response_text = response_text[:context.max_tokens * 4] + "..."
        
        routing_time = time.time() - start_time
        
        return {
            "query": query,
            "response": response_text,
            "analysis": analysis,
            "context": {
                "expertise": context.expertise_level.value,
                "format": context.format_preference.value
            },
            "routing_time": routing_time,
            "chain_used": chain.__class__.__name__
        }

# Test the intelligent router
print("Intelligent Routing System Test\n")
print("=" * 50)

router = IntelligentRouter()

# Test scenarios
test_scenarios = [
    {
        "query": "How do I write a for loop?",
        "context": UserContext(
            expertise_level=ExpertiseLevel.BEGINNER,
            format_preference=FormatPreference.BRIEF
        )
    },
    {
        "query": "Explain the architecture of a microservices system with examples",
        "context": UserContext(
            expertise_level=ExpertiseLevel.EXPERT,
            format_preference=FormatPreference.STRUCTURED
        )
    },
    {
        "query": "What's the ROI of digital transformation?",
        "context": UserContext(
            expertise_level=ExpertiseLevel.INTERMEDIATE,
            format_preference=FormatPreference.DETAILED
        )
    },
    {
        "query": "Write a creative story about a robot",
        "context": UserContext(
            expertise_level=ExpertiseLevel.INTERMEDIATE,
            format_preference=FormatPreference.BRIEF
        )
    }
]

for scenario in test_scenarios:
    result = router.route(scenario["query"], scenario["context"])
    
    print(f"\nQuery: {result['query']}")
    print(f"Analysis: Domain={result['analysis']['domain']}, Complexity={result['analysis']['complexity']}")
    print(f"Context: {result['context']}")
    print(f"Routing Time: {result['routing_time']:.3f}s")
    print(f"Response Preview: {result['response'][:150]}...")
    print("-" * 50)
"""

print("Build an intelligent multi-dimensional router!")
print("The solution includes query analysis, context awareness, and fallback handling.")

---

## **Instructor Activity 2: Retry, Fallback, and Self-Correction**

Let's implement robust error handling and self-correction patterns:

In [ ]:
# Advanced retry with exponential backoff

class RetryableChain:
    """Chain with sophisticated retry logic."""
    
    def __init__(self, max_retries: int = 3):
        self.max_retries = max_retries
        self.llm = ChatOpenAI(temperature=0.7)
        self.retry_history = []
    
    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=1, max=10)
    )
    def _execute_with_retry(self, prompt: str) -> str:
        """Execute with exponential backoff retry."""
        # Simulate occasional failures
        if random.random() < 0.3:  # 30% failure rate for demo
            raise Exception("Simulated API error")
        
        response = self.llm.predict(prompt)
        return response
    
    def execute(self, prompt: str) -> Dict:
        """Execute with comprehensive error handling."""
        attempt = 0
        errors = []
        
        while attempt < self.max_retries:
            try:
                attempt += 1
                print(f"Attempt {attempt}/{self.max_retries}...")
                
                # Add retry context to prompt if not first attempt
                if attempt > 1:
                    retry_context = f"\n\n[Previous attempt failed. Please try again carefully.]"
                    enhanced_prompt = prompt + retry_context
                else:
                    enhanced_prompt = prompt
                
                result = self._execute_with_retry(enhanced_prompt)
                
                self.retry_history.append({
                    "attempt": attempt,
                    "success": True,
                    "result": result
                })
                
                return {
                    "success": True,
                    "result": result,
                    "attempts": attempt,
                    "errors": errors
                }
                
            except Exception as e:
                errors.append({
                    "attempt": attempt,
                    "error": str(e),
                    "timestamp": time.time()
                })
                
                self.retry_history.append({
                    "attempt": attempt,
                    "success": False,
                    "error": str(e)
                })
                
                if attempt < self.max_retries:
                    wait_time = 2 ** attempt  # Exponential backoff
                    print(f"  Error: {e}. Waiting {wait_time}s before retry...")
                    time.sleep(wait_time)
        
        # All retries failed
        return {
            "success": False,
            "result": None,
            "attempts": attempt,
            "errors": errors
        }

# Test retry chain
retry_chain = RetryableChain(max_retries=3)

result = retry_chain.execute("What is the capital of Japan?")

print("\nResult:")
print(f"Success: {result['success']}")
print(f"Attempts: {result['attempts']}")
if result['success']:
    print(f"Answer: {result['result']}")
else:
    print(f"Errors: {result['errors']}")

In [ ]:
# Self-correcting chain with validation

class SelfCorrectingChain:
    """Chain that validates and corrects its own output."""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.7)
        self.validator_llm = ChatOpenAI(temperature=0)  # More deterministic for validation
    
    def generate(self, prompt: str) -> str:
        """Generate initial response."""
        return self.llm.predict(prompt)
    
    def validate(self, prompt: str, response: str) -> Dict:
        """Validate the response."""
        validation_prompt = f"""Validate this response for accuracy and completeness.
        
        Original Question: {prompt}
        Response: {response}
        
        Check for:
        1. Factual accuracy
        2. Completeness
        3. Clarity
        4. Relevance
        
        Return JSON:
        {{
            "is_valid": true/false,
            "issues": [list of issues if any],
            "suggestions": [list of improvements]
        }}
        
        JSON:"""
        
        validation_result = self.validator_llm.predict(validation_prompt)
        
        try:
            return json.loads(validation_result)
        except:
            # Default to valid if parsing fails
            return {"is_valid": True, "issues": [], "suggestions": []}
    
    def correct(self, prompt: str, response: str, validation: Dict) -> str:
        """Correct the response based on validation."""
        correction_prompt = f"""Improve this response based on the feedback.
        
        Original Question: {prompt}
        Original Response: {response}
        
        Issues to fix: {validation['issues']}
        Suggestions: {validation['suggestions']}
        
        Improved Response:"""
        
        return self.llm.predict(correction_prompt)
    
    def execute_with_correction(self, prompt: str, max_corrections: int = 2) -> Dict:
        """Execute with self-correction loop."""
        correction_count = 0
        history = []
        
        # Initial generation
        response = self.generate(prompt)
        history.append({"version": 0, "response": response})
        
        while correction_count < max_corrections:
            # Validate
            validation = self.validate(prompt, response)
            
            if validation["is_valid"] and not validation["issues"]:
                # Response is valid
                return {
                    "final_response": response,
                    "corrections_made": correction_count,
                    "history": history,
                    "validation": validation
                }
            
            # Need correction
            correction_count += 1
            print(f"  Correction {correction_count}: Fixing issues {validation['issues']}")
            
            response = self.correct(prompt, response, validation)
            history.append({
                "version": correction_count,
                "response": response,
                "fixed_issues": validation["issues"]
            })
        
        # Max corrections reached
        return {
            "final_response": response,
            "corrections_made": correction_count,
            "history": history,
            "validation": validation
        }

# Test self-correcting chain
self_correcting = SelfCorrectingChain()

test_prompts = [
    "Write a Python function to calculate factorial",
    "Explain the water cycle with all major steps"
]

for prompt in test_prompts:
    print(f"\nPrompt: {prompt}")
    print("Processing with self-correction...")
    
    result = self_correcting.execute_with_correction(prompt)
    
    print(f"\nCorrections made: {result['corrections_made']}")
    print(f"Final Response: {result['final_response'][:200]}...")
    
    if result['corrections_made'] > 0:
        print("\nCorrection History:")
        for item in result['history'][1:]:  # Skip original
            print(f"  Version {item['version']}: Fixed {item.get('fixed_issues', [])}")
    
    print("=" * 50)

---

## **Learner Activity 2: Build a Resilient Chain System**

Create a comprehensive resilient system with multiple fallback strategies.

**Task**: Build a system that:
1. Has primary, secondary, and tertiary processing chains
2. Implements intelligent fallback selection
3. Includes self-healing capabilities
4. Tracks and learns from failures
5. Provides graceful degradation

Requirements:
- Multiple fallback strategies
- Failure pattern recognition
- Adaptive retry logic
- Performance metrics tracking

In [ ]:
# Build your resilient chain system

class ResilientSystem:
    def __init__(self):
        # TODO: Initialize primary, secondary, tertiary chains
        # TODO: Setup failure tracking
        pass
    
    def execute_with_fallback(self, prompt: str) -> Dict:
        """Execute with intelligent fallback."""
        # TODO: Try primary chain
        # TODO: On failure, select appropriate fallback
        # TODO: Track failure patterns
        pass
    
    def learn_from_failure(self, failure_info: Dict):
        """Learn from failures to improve future routing."""
        # TODO: Analyze failure patterns
        # TODO: Adjust routing strategy
        pass
    
    def get_health_status(self) -> Dict:
        """Get system health and performance metrics."""
        # TODO: Return health metrics
        pass

# TODO: Test resilient system
# Simulate various failure scenarios
# Verify fallback behavior
# Check learning from failures

# Your test code here

In [ ]:
# Solution (hidden by default)

"""
from collections import defaultdict, deque
from datetime import datetime, timedelta

class ChainHealth:
    '''Track health metrics for a chain.'''
    def __init__(self, window_size: int = 100):
        self.success_count = 0
        self.failure_count = 0
        self.recent_latencies = deque(maxlen=window_size)
        self.recent_errors = deque(maxlen=window_size)
        self.last_failure_time = None
    
    @property
    def success_rate(self) -> float:
        total = self.success_count + self.failure_count
        return self.success_count / total if total > 0 else 1.0
    
    @property
    def avg_latency(self) -> float:
        return sum(self.recent_latencies) / len(self.recent_latencies) if self.recent_latencies else 0
    
    @property
    def health_score(self) -> float:
        '''Composite health score 0-1.'''
        # Weight factors
        success_weight = 0.5
        latency_weight = 0.3
        recency_weight = 0.2
        
        # Success component
        success_score = self.success_rate
        
        # Latency component (normalize to 0-1, assuming 5s is bad)
        latency_score = max(0, 1 - (self.avg_latency / 5)) if self.recent_latencies else 1
        
        # Recency component (penalize recent failures)
        if self.last_failure_time:
            time_since_failure = (datetime.now() - self.last_failure_time).seconds
            recency_score = min(1, time_since_failure / 300)  # 5 min to full recovery
        else:
            recency_score = 1
        
        return (
            success_weight * success_score +
            latency_weight * latency_score +
            recency_weight * recency_score
        )

class ResilientSystem:
    def __init__(self):
        # Initialize chains with different models/configs
        self.primary_chain = self._create_chain("gpt-3.5-turbo", 0.7)
        self.secondary_chain = self._create_chain("gpt-3.5-turbo", 0.5)  # Lower temp
        self.tertiary_chain = self._create_chain("gpt-3.5-turbo", 0.3)  # Even lower
        
        # Health tracking
        self.chain_health = {
            "primary": ChainHealth(),
            "secondary": ChainHealth(),
            "tertiary": ChainHealth()
        }
        
        # Failure patterns
        self.failure_patterns = defaultdict(list)
        self.adaptive_routing = True
        
        # Circuit breaker settings
        self.circuit_breaker = {
            "primary": "closed",  # closed, open, half_open
            "secondary": "closed",
            "tertiary": "closed"
        }
        self.circuit_breaker_threshold = 0.5  # Open if success rate < 50%
        self.circuit_breaker_timeout = 30  # Seconds before trying half_open
    
    def _create_chain(self, model: str, temperature: float):
        '''Create a chain with specific configuration.'''
        llm = ChatOpenAI(model=model, temperature=temperature)
        prompt = ChatPromptTemplate.from_template("Answer this: {prompt}")
        return prompt | llm
    
    def _check_circuit_breaker(self, chain_name: str) -> bool:
        '''Check if circuit breaker allows execution.'''
        state = self.circuit_breaker[chain_name]
        health = self.chain_health[chain_name]
        
        if state == "closed":
            # Check if should open
            if health.success_rate < self.circuit_breaker_threshold and health.failure_count > 5:
                self.circuit_breaker[chain_name] = "open"
                print(f"  🔴 Circuit breaker OPENED for {chain_name}")
                return False
            return True
        
        elif state == "open":
            # Check if should try half_open
            if health.last_failure_time:
                time_since = (datetime.now() - health.last_failure_time).seconds
                if time_since > self.circuit_breaker_timeout:
                    self.circuit_breaker[chain_name] = "half_open"
                    print(f"  🟡 Circuit breaker HALF-OPEN for {chain_name}")
                    return True
            return False
        
        else:  # half_open
            return True
    
    def _execute_chain(self, chain_name: str, chain, prompt: str) -> tuple:
        '''Execute a chain with health tracking.'''
        health = self.chain_health[chain_name]
        
        # Check circuit breaker
        if not self._check_circuit_breaker(chain_name):
            return False, "Circuit breaker open"
        
        start_time = time.time()
        
        try:
            # Simulate occasional failures for demo
            if chain_name == "primary" and random.random() < 0.4:  # 40% failure rate
                raise Exception("Simulated primary chain failure")
            elif chain_name == "secondary" and random.random() < 0.2:  # 20% failure rate
                raise Exception("Simulated secondary chain failure")
            
            result = chain.invoke({"prompt": prompt})
            latency = time.time() - start_time
            
            # Update health
            health.success_count += 1
            health.recent_latencies.append(latency)
            
            # Close circuit breaker if half_open
            if self.circuit_breaker[chain_name] == "half_open":
                self.circuit_breaker[chain_name] = "closed"
                print(f"  🟢 Circuit breaker CLOSED for {chain_name}")
            
            return True, result.content
            
        except Exception as e:
            latency = time.time() - start_time
            
            # Update health
            health.failure_count += 1
            health.recent_latencies.append(latency)
            health.recent_errors.append(str(e))
            health.last_failure_time = datetime.now()
            
            # Open circuit breaker if half_open
            if self.circuit_breaker[chain_name] == "half_open":
                self.circuit_breaker[chain_name] = "open"
                print(f"  🔴 Circuit breaker RE-OPENED for {chain_name}")
            
            return False, str(e)
    
    def execute_with_fallback(self, prompt: str) -> Dict:
        '''Execute with intelligent fallback.'''
        execution_log = []
        start_time = time.time()
        
        # Determine chain order based on health scores
        if self.adaptive_routing:
            chain_order = self._get_adaptive_order()
        else:
            chain_order = [("primary", self.primary_chain),
                         ("secondary", self.secondary_chain),
                         ("tertiary", self.tertiary_chain)]
        
        # Try chains in order
        for chain_name, chain in chain_order:
            print(f"\n  Trying {chain_name} chain...")
            success, result = self._execute_chain(chain_name, chain, prompt)
            
            execution_log.append({
                "chain": chain_name,
                "success": success,
                "result": result if success else None,
                "error": result if not success else None
            })
            
            if success:
                total_time = time.time() - start_time
                return {
                    "success": True,
                    "result": result,
                    "chain_used": chain_name,
                    "execution_log": execution_log,
                    "total_time": total_time
                }
        
        # All chains failed
        self._record_failure_pattern(prompt, execution_log)
        
        # Last resort: return cached or default response
        fallback_response = self._get_fallback_response(prompt)
        
        return {
            "success": False,
            "result": fallback_response,
            "chain_used": "fallback",
            "execution_log": execution_log,
            "total_time": time.time() - start_time
        }
    
    def _get_adaptive_order(self) -> list:
        '''Get chain order based on health scores.'''
        chains_with_scores = [
            ("primary", self.primary_chain, self.chain_health["primary"].health_score),
            ("secondary", self.secondary_chain, self.chain_health["secondary"].health_score),
            ("tertiary", self.tertiary_chain, self.chain_health["tertiary"].health_score)
        ]
        
        # Sort by health score (best first)
        chains_with_scores.sort(key=lambda x: x[2], reverse=True)
        
        return [(name, chain) for name, chain, _ in chains_with_scores]
    
    def _record_failure_pattern(self, prompt: str, execution_log: list):
        '''Record failure pattern for learning.'''
        pattern = {
            "prompt_length": len(prompt),
            "prompt_type": self._classify_prompt(prompt),
            "failures": [log["chain"] for log in execution_log if not log["success"]],
            "timestamp": datetime.now()
        }
        
        self.failure_patterns[pattern["prompt_type"]].append(pattern)
    
    def _classify_prompt(self, prompt: str) -> str:
        '''Simple prompt classification.'''
        if "?" in prompt:
            return "question"
        elif any(word in prompt.lower() for word in ["create", "write", "generate"]):
            return "generation"
        elif any(word in prompt.lower() for word in ["analyze", "explain", "describe"]):
            return "analysis"
        else:
            return "other"
    
    def _get_fallback_response(self, prompt: str) -> str:
        '''Get fallback response when all chains fail.'''
        prompt_type = self._classify_prompt(prompt)
        
        fallback_responses = {
            "question": "I'm unable to answer your question at the moment. Please try again later.",
            "generation": "I'm unable to generate content right now. Please try again.",
            "analysis": "I'm unable to perform the analysis currently. Please retry.",
            "other": "I'm experiencing technical difficulties. Please try again later."
        }
        
        return fallback_responses.get(prompt_type, fallback_responses["other"])
    
    def learn_from_failure(self, prompt_type: str = None):
        '''Analyze failure patterns and adjust strategy.'''
        if prompt_type and prompt_type in self.failure_patterns:
            patterns = self.failure_patterns[prompt_type]
            
            if len(patterns) > 5:  # Need enough data
                # Find most common failing chain
                failing_chains = []
                for pattern in patterns[-10:]:  # Last 10 failures
                    failing_chains.extend(pattern["failures"])
                
                from collections import Counter
                chain_failures = Counter(failing_chains)
                
                # Adjust routing if clear pattern
                most_failed = chain_failures.most_common(1)[0] if chain_failures else None
                if most_failed and most_failed[1] > 5:
                    print(f"  📊 Learning: {most_failed[0]} chain frequently fails for {prompt_type}")
                    # Could adjust routing strategy here
    
    def get_health_status(self) -> Dict:
        '''Get comprehensive health status.'''
        status = {
            "chains": {},
            "circuit_breakers": self.circuit_breaker.copy(),
            "failure_patterns": {k: len(v) for k, v in self.failure_patterns.items()},
            "adaptive_routing": self.adaptive_routing
        }
        
        for chain_name, health in self.chain_health.items():
            status["chains"][chain_name] = {
                "health_score": health.health_score,
                "success_rate": health.success_rate,
                "avg_latency": health.avg_latency,
                "total_requests": health.success_count + health.failure_count
            }
        
        return status

# Test the resilient system
print("Resilient System Test\n")
print("=" * 50)

system = ResilientSystem()

# Test with multiple queries
test_prompts = [
    "What is artificial intelligence?",
    "Write a haiku about coding",
    "Explain quantum computing",
    "How does photosynthesis work?",
    "Generate a business plan outline"
]

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n[{i}] Query: {prompt}")
    result = system.execute_with_fallback(prompt)
    
    print(f"\n  Result:")
    print(f"    Success: {result['success']}")
    print(f"    Chain Used: {result['chain_used']}")
    print(f"    Time: {result['total_time']:.2f}s")
    
    if result['success']:
        print(f"    Response: {result['result'][:100]}...")
    
    # Show execution path
    print(f"\n  Execution Path:")
    for log in result['execution_log']:
        status = "✅" if log['success'] else "❌"
        print(f"    {status} {log['chain']}")

# Show health status
print("\n" + "=" * 50)
print("\n📊 System Health Status:")
health = system.get_health_status()

for chain_name, metrics in health['chains'].items():
    print(f"\n  {chain_name.capitalize()} Chain:")
    print(f"    Health Score: {metrics['health_score']:.2f}")
    print(f"    Success Rate: {metrics['success_rate']:.1%}")
    print(f"    Avg Latency: {metrics['avg_latency']:.2f}s")
    print(f"    Circuit Breaker: {health['circuit_breakers'][chain_name]}")

# Learn from failures
print("\n📚 Learning from failures...")
system.learn_from_failure("question")
"""

print("Build a comprehensive resilient system with fallbacks and self-healing!")
print("The solution includes circuit breakers, adaptive routing, and failure learning.")

---

## **Instructor Activity 3: Map-Reduce and Complex Orchestration**

Let's implement advanced orchestration patterns:

In [ ]:
# Map-Reduce pattern for processing large documents

class MapReduceProcessor:
    """Process large documents using map-reduce pattern."""
    
    def __init__(self, chunk_size: int = 1000):
        self.chunk_size = chunk_size
        self.llm = ChatOpenAI(temperature=0.7)
    
    def split_document(self, document: str) -> List[str]:
        """Split document into chunks."""
        words = document.split()
        chunks = []
        
        for i in range(0, len(words), self.chunk_size // 5):  # Rough word count
            chunk = " ".join(words[i:i + self.chunk_size // 5])
            if chunk:
                chunks.append(chunk)
        
        return chunks
    
    def map_operation(self, chunk: str, operation: str) -> str:
        """Apply operation to a single chunk."""
        map_prompt = f"""{operation}
        
        Text: {chunk}
        
        Result:"""
        
        result = self.llm.predict(map_prompt)
        return result
    
    def reduce_operation(self, results: List[str], operation: str) -> str:
        """Reduce/combine results from map phase."""
        reduce_prompt = f"""{operation}
        
        Individual Results:
        {chr(10).join(f'{i+1}. {r}' for i, r in enumerate(results))}
        
        Combined Result:"""
        
        final_result = self.llm.predict(reduce_prompt)
        return final_result
    
    def process(self, document: str, map_instruction: str, reduce_instruction: str) -> Dict:
        """Execute map-reduce operation."""
        # Split
        chunks = self.split_document(document)
        print(f"Split document into {len(chunks)} chunks")
        
        # Map phase
        print("\nMap phase:")
        map_results = []
        for i, chunk in enumerate(chunks):
            print(f"  Processing chunk {i+1}/{len(chunks)}...")
            result = self.map_operation(chunk, map_instruction)
            map_results.append(result)
        
        # Reduce phase
        print("\nReduce phase:")
        final_result = self.reduce_operation(map_results, reduce_instruction)
        
        return {
            "chunks_processed": len(chunks),
            "map_results": map_results,
            "final_result": final_result
        }

# Test map-reduce
processor = MapReduceProcessor(chunk_size=500)

# Sample document
document = """Artificial intelligence has revolutionized many industries. 
In healthcare, AI helps diagnose diseases and develop new treatments.
In finance, AI detects fraud and manages investments.
In transportation, AI powers self-driving cars.
However, AI also raises ethical concerns about privacy and job displacement.
We must ensure AI is developed responsibly and benefits everyone."""

result = processor.process(
    document=document,
    map_instruction="Extract the main point from this text segment",
    reduce_instruction="Combine these points into a comprehensive summary"
)

print("\n" + "=" * 50)
print("\nMap Results:")
for i, mr in enumerate(result["map_results"]):
    print(f"{i+1}. {mr}")

print("\nFinal Result:")
print(result["final_result"])

---

## **Learner Activity 3: Build an Advanced Orchestration System**

Create a sophisticated system that orchestrates multiple processing patterns.

**Task**: Build a system that:
1. Supports map-reduce, pipeline, and parallel processing
2. Dynamically selects processing strategy
3. Handles nested orchestrations
4. Provides progress tracking
5. Optimizes for cost and performance

Requirements:
- Multiple orchestration patterns
- Dynamic strategy selection
- Resource optimization
- Comprehensive monitoring

In [ ]:
# Build your advanced orchestration system

class OrchestrationStrategy(Enum):
    MAP_REDUCE = "map_reduce"
    PIPELINE = "pipeline"
    PARALLEL = "parallel"
    HYBRID = "hybrid"

class AdvancedOrchestrator:
    def __init__(self):
        # TODO: Initialize orchestration components
        pass
    
    def select_strategy(self, task: Dict) -> OrchestrationStrategy:
        """Select optimal orchestration strategy."""
        # TODO: Analyze task and select strategy
        pass
    
    def execute_map_reduce(self, task: Dict) -> Dict:
        """Execute using map-reduce pattern."""
        # TODO: Implement map-reduce execution
        pass
    
    def execute_pipeline(self, task: Dict) -> Dict:
        """Execute using pipeline pattern."""
        # TODO: Implement pipeline execution
        pass
    
    def execute_parallel(self, task: Dict) -> Dict:
        """Execute using parallel pattern."""
        # TODO: Implement parallel execution
        pass
    
    def orchestrate(self, task: Dict) -> Dict:
        """Main orchestration method."""
        # TODO: Select and execute strategy
        # TODO: Track progress and optimize
        pass

# TODO: Test orchestration system
# Test different task types
# Verify strategy selection
# Check performance optimization

# Your test code here

In [ ]:
# Solution (hidden by default)

"""
import asyncio
from concurrent.futures import ThreadPoolExecutor
import time

class AdvancedOrchestrator:
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.7)
        self.executor = ThreadPoolExecutor(max_workers=5)
        self.cost_tracker = {"tokens": 0, "requests": 0}
        self.performance_metrics = []
    
    def select_strategy(self, task: Dict) -> OrchestrationStrategy:
        '''Select optimal orchestration strategy based on task characteristics.'''
        
        task_type = task.get("type", "general")
        data_size = len(task.get("data", ""))
        complexity = task.get("complexity", "medium")
        
        # Strategy selection logic
        if task_type == "aggregation" or data_size > 5000:
            return OrchestrationStrategy.MAP_REDUCE
        elif task_type == "sequential" or "steps" in task:
            return OrchestrationStrategy.PIPELINE
        elif task_type == "independent" or task.get("parallelize", False):
            return OrchestrationStrategy.PARALLEL
        else:
            # Hybrid for complex tasks
            if complexity == "high":
                return OrchestrationStrategy.HYBRID
            else:
                return OrchestrationStrategy.PIPELINE
    
    def execute_map_reduce(self, task: Dict) -> Dict:
        '''Execute using map-reduce pattern.'''
        print("\n🗺️  Executing MAP-REDUCE strategy")
        
        data = task.get("data", "")
        map_fn = task.get("map_function", "Summarize this chunk")
        reduce_fn = task.get("reduce_function", "Combine these summaries")
        
        # Split data
        chunks = self._split_data(data, task.get("chunk_size", 1000))
        print(f"  Split into {len(chunks)} chunks")
        
        # Map phase (parallel)
        print("  Map phase...")
        map_results = []
        
        with ThreadPoolExecutor(max_workers=3) as executor:
            futures = []
            for chunk in chunks:
                future = executor.submit(self._process_chunk, chunk, map_fn)
                futures.append(future)
            
            for future in futures:
                result = future.result()
                map_results.append(result)
        
        # Reduce phase
        print("  Reduce phase...")
        reduce_prompt = f"{reduce_fn}\n\nResults:\n" + "\n".join(map_results)
        final_result = self.llm.predict(reduce_prompt)
        
        return {
            "strategy": "map_reduce",
            "chunks_processed": len(chunks),
            "result": final_result
        }
    
    def execute_pipeline(self, task: Dict) -> Dict:
        '''Execute using pipeline pattern.'''
        print("\n🚇 Executing PIPELINE strategy")
        
        data = task.get("data", "")
        steps = task.get("steps", [])
        
        if not steps:
            # Default pipeline
            steps = [
                "Analyze the input",
                "Process and transform",
                "Generate final output"
            ]
        
        current_data = data
        pipeline_results = []
        
        for i, step in enumerate(steps, 1):
            print(f"  Step {i}/{len(steps)}: {step[:30]}...")
            
            prompt = f"{step}\n\nInput: {current_data}\n\nOutput:"
            result = self.llm.predict(prompt)
            
            pipeline_results.append({
                "step": i,
                "instruction": step,
                "output": result
            })
            
            current_data = result  # Feed to next step
        
        return {
            "strategy": "pipeline",
            "steps_executed": len(steps),
            "pipeline_results": pipeline_results,
            "result": current_data
        }
    
    def execute_parallel(self, task: Dict) -> Dict:
        '''Execute using parallel pattern.'''
        print("\n⚡ Executing PARALLEL strategy")
        
        data = task.get("data", "")
        operations = task.get("operations", [])
        
        if not operations:
            # Default parallel operations
            operations = [
                "Provide a summary",
                "Extract key points",
                "Identify main themes"
            ]
        
        print(f"  Running {len(operations)} operations in parallel")
        
        # Execute all operations in parallel
        with ThreadPoolExecutor(max_workers=len(operations)) as executor:
            futures = []
            for op in operations:
                prompt = f"{op}\n\nText: {data}\n\nResult:"
                future = executor.submit(self.llm.predict, prompt)
                futures.append((op, future))
            
            results = []
            for op, future in futures:
                result = future.result()
                results.append({
                    "operation": op,
                    "result": result
                })
        
        # Combine results
        combined = "\n\n".join([f"{r['operation']}:\n{r['result']}" for r in results])
        
        return {
            "strategy": "parallel",
            "operations_count": len(operations),
            "parallel_results": results,
            "result": combined
        }
    
    def execute_hybrid(self, task: Dict) -> Dict:
        '''Execute using hybrid pattern (map-reduce + pipeline).'''
        print("\n🔄 Executing HYBRID strategy")
        
        # Phase 1: Map-Reduce for initial processing
        print("  Phase 1: Map-Reduce")
        map_reduce_task = {
            "data": task.get("data", ""),
            "map_function": "Extract important information",
            "reduce_function": "Synthesize the extracted information"
        }
        mr_result = self.execute_map_reduce(map_reduce_task)
        
        # Phase 2: Pipeline for refinement
        print("\n  Phase 2: Pipeline")
        pipeline_task = {
            "data": mr_result["result"],
            "steps": [
                "Enhance and clarify the content",
                "Add structure and formatting",
                "Polish and finalize"
            ]
        }
        pipeline_result = self.execute_pipeline(pipeline_task)
        
        return {
            "strategy": "hybrid",
            "phases": ["map_reduce", "pipeline"],
            "result": pipeline_result["result"]
        }
    
    def _split_data(self, data: str, chunk_size: int) -> List[str]:
        '''Split data into chunks.'''
        words = data.split()
        chunks = []
        
        for i in range(0, len(words), chunk_size // 5):
            chunk = " ".join(words[i:i + chunk_size // 5])
            if chunk:
                chunks.append(chunk)
        
        return chunks
    
    def _process_chunk(self, chunk: str, instruction: str) -> str:
        '''Process a single chunk.'''
        prompt = f"{instruction}\n\nChunk: {chunk}\n\nResult:"
        return self.llm.predict(prompt)
    
    def orchestrate(self, task: Dict) -> Dict:
        '''Main orchestration method with optimization.'''
        start_time = time.time()
        
        # Select strategy
        strategy = self.select_strategy(task)
        print(f"\n🎯 Selected Strategy: {strategy.value}")
        
        # Track initial cost
        initial_tokens = self.cost_tracker["tokens"]
        
        # Execute based on strategy
        try:
            if strategy == OrchestrationStrategy.MAP_REDUCE:
                result = self.execute_map_reduce(task)
            elif strategy == OrchestrationStrategy.PIPELINE:
                result = self.execute_pipeline(task)
            elif strategy == OrchestrationStrategy.PARALLEL:
                result = self.execute_parallel(task)
            else:  # HYBRID
                result = self.execute_hybrid(task)
            
            # Calculate metrics
            execution_time = time.time() - start_time
            
            # Estimate tokens (simplified)
            estimated_tokens = len(str(result)) // 4
            self.cost_tracker["tokens"] += estimated_tokens
            self.cost_tracker["requests"] += 1
            
            # Add performance metrics
            metrics = {
                "strategy": strategy.value,
                "execution_time": execution_time,
                "estimated_tokens": estimated_tokens,
                "estimated_cost": estimated_tokens * 0.000002  # Example pricing
            }
            
            self.performance_metrics.append(metrics)
            result["metrics"] = metrics
            
            print(f"\n📊 Execution Metrics:")
            print(f"  Time: {execution_time:.2f}s")
            print(f"  Estimated Tokens: {estimated_tokens}")
            print(f"  Estimated Cost: ${metrics['estimated_cost']:.4f}")
            
            return result
            
        except Exception as e:
            return {
                "strategy": strategy.value,
                "error": str(e),
                "result": None
            }
    
    def get_optimization_report(self) -> Dict:
        '''Get optimization and performance report.'''
        if not self.performance_metrics:
            return {"message": "No metrics available"}
        
        total_time = sum(m["execution_time"] for m in self.performance_metrics)
        total_cost = sum(m["estimated_cost"] for m in self.performance_metrics)
        
        # Strategy distribution
        from collections import Counter
        strategy_counts = Counter(m["strategy"] for m in self.performance_metrics)
        
        return {
            "total_executions": len(self.performance_metrics),
            "total_time": total_time,
            "total_estimated_cost": total_cost,
            "avg_execution_time": total_time / len(self.performance_metrics),
            "strategy_distribution": dict(strategy_counts),
            "total_tokens": self.cost_tracker["tokens"],
            "recommendations": self._get_optimization_recommendations()
        }
    
    def _get_optimization_recommendations(self) -> List[str]:
        '''Generate optimization recommendations.'''
        recommendations = []
        
        if self.performance_metrics:
            avg_time = sum(m["execution_time"] for m in self.performance_metrics) / len(self.performance_metrics)
            
            if avg_time > 5:
                recommendations.append("Consider increasing parallelization for better performance")
            
            # Check strategy efficiency
            strategy_times = {}
            for m in self.performance_metrics:
                if m["strategy"] not in strategy_times:
                    strategy_times[m["strategy"]] = []
                strategy_times[m["strategy"]].append(m["execution_time"])
            
            for strategy, times in strategy_times.items():
                avg_strategy_time = sum(times) / len(times)
                if avg_strategy_time > 10:
                    recommendations.append(f"Optimize {strategy} strategy - avg time {avg_strategy_time:.1f}s")
        
        if self.cost_tracker["tokens"] > 100000:
            recommendations.append("High token usage - consider caching or compression")
        
        return recommendations if recommendations else ["System operating efficiently"]

# Test the orchestrator
print("Advanced Orchestration System Test")
print("=" * 50)

orchestrator = AdvancedOrchestrator()

# Test different task types
test_tasks = [
    {
        "name": "Large Document Summary",
        "type": "aggregation",
        "data": "AI is transforming industries. " * 100,  # Large text
        "complexity": "high"
    },
    {
        "name": "Sequential Processing",
        "type": "sequential",
        "data": "Raw data that needs processing",
        "steps": [
            "Clean and normalize the data",
            "Extract meaningful patterns",
            "Generate insights"
        ]
    },
    {
        "name": "Parallel Analysis",
        "type": "independent",
        "data": "Complex topic requiring multiple perspectives",
        "parallelize": True,
        "operations": [
            "Technical analysis",
            "Business perspective",
            "User experience view"
        ]
    },
    {
        "name": "Complex Hybrid Task",
        "type": "complex",
        "data": "Multi-faceted problem requiring sophisticated processing",
        "complexity": "high"
    }
]

for task in test_tasks:
    print(f"\n{'='*50}")
    print(f"Task: {task['name']}")
    
    result = orchestrator.orchestrate(task)
    
    print(f"\nResult Preview:")
    if result.get("result"):
        print(f"  {result['result'][:200]}...")

# Show optimization report
print("\n" + "="*50)
print("\n📈 Optimization Report:")
report = orchestrator.get_optimization_report()

for key, value in report.items():
    if key != "recommendations":
        print(f"  {key}: {value}")

print("\n  Recommendations:")
for rec in report["recommendations"]:
    print(f"    • {rec}")
"""

print("Build an advanced orchestration system with multiple processing patterns!")
print("The solution includes map-reduce, pipeline, parallel, and hybrid strategies.")

---

## **Summary and Next Steps**

Congratulations! You've mastered advanced LangChain patterns. You can now:

✅ Implement sophisticated routing and conditional logic
✅ Build resilient systems with retry and fallback mechanisms
✅ Create self-correcting chains
✅ Design map-reduce processing patterns
✅ Orchestrate complex multi-pattern workflows
✅ Optimize for performance and cost

### **Key Takeaways:**
- **Routing Enables Flexibility**: Dynamic routing adapts to different inputs
- **Resilience is Critical**: Production systems need fallbacks and retries
- **Self-Correction Improves Quality**: Validation and correction loops enhance output
- **Orchestration Scales**: Map-reduce and parallel patterns handle large workloads
- **Optimization Matters**: Track metrics and optimize strategies

### **Next Steps:**
- **Notebook 12**: Learn about Production Deployment
- **Practice**: Implement these patterns in real applications
- **Experiment**: Combine patterns for complex use cases
- **Scale**: Deploy these patterns in production

### **Additional Challenges:**
1. Build a self-optimizing routing system that learns from usage
2. Create a distributed map-reduce system across multiple servers
3. Implement a circuit breaker pattern with adaptive thresholds
4. Design a cost-optimized orchestrator for different LLM providers
5. Build a meta-orchestrator that can compose orchestration patterns